<h3> Palabras (TF-IDF)</h3>
<ol><li>Calcular el promedio de TF-IDF para cada palabra en el clúster
        tfidf_promedio = cluster_tfidf.mean(axis=0).A1  #A1 convierte la matriz sparse a un array 1D eficiente</li>
<li> los índices de las n palabras con la mayor puntuación TF-IDF promedio
        palabras_representativas_indices = np.argsort(tfidf_promedio)[::-1][:n]</li></ol>
<h3>Documentos (TF-IDF) a nivel de clúster</h3>
<ol><li>Calcular Tf a nivel de clúster</li>
<li>Calcular IDF a nivel de clúster</li>
<li>Generar puntuaciones para cada documento del clúster </li></ol>
<h3> Palabras (Word2Vec)</h3>
<ol><li>Obtener embeddings promedio de palabras por documento
        cluster_vectores = []
        for doc in cluster_documentos:
            palabra_vectors = [model_w2v.wv[palabra] for palabra in doc if palabra in model_w2v.wv]
            if palabra_vectors:  # Evitar documentos vacíos
                cluster_vectores.append(np.mean(palabra_vectors, axis=0))</li>

<li> Calcular el promedio del clúster
        cluster_centroide = np.mean(cluster_vectores, axis=0)</li>
<li> Encontrar palabras más similar al promedio
        palabras_cercanas = model_w2v.wv.similar_by_vector(cluster_centroide, topn=n)</li></ol>
<h3> Documentos (Word2Vec)</h3>
    <ol><li> Aplicar la misma función que para TF-IDF</li>
</ol>

In [1]:
import pandas as pd
import pickle
import bz2
from IPython.display import display, HTML
import numpy as np
import ast
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
with open('recursos/resultados_modelos.pkl', 'rb') as archivo:
    resultados = pickle.load(archivo)

In [3]:
etiquetas_predichas=resultados["agglomerative_Tfidf"]['etiquetas_predichas'][0]

In [4]:
with open('recursos/modelos_topicos/agglomerative_Tfidf_ejecucion_0.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)

In [5]:
def filter_documents_for_dictionary(tokenized_documents, filtered_words):
    filtered_documents = []
    for doc in tokenized_documents:
        filtered_doc = [word for word in doc if word in filtered_words]
        filtered_documents.append(filtered_doc)
    return filtered_documents

In [6]:
#df_preprocesado = pd.read_csv('df_preprocesado.csv', sep=';')
dictionary = Dictionary.load('recursos/dictionary_filtrado_spacy.gensim')
filtered_words = set(dictionary.token2id.keys())
n_term=len(filtered_words)
#preprocesado_documents=df_preprocesado['texto_preprocesado'].apply(ast.literal_eval)    
#documents = filter_documents_for_dictionary(preprocesado_documents, filtered_words) 
with open('recursos/documents.pkl', 'rb') as archivo:
    documents = pickle.load(archivo)

In [7]:
documentos_como_cadenas = [" ".join(doc) for doc in documents]

In [8]:
vectorizer = TfidfVectorizer()
matriz_tfidf = vectorizer.fit_transform(documentos_como_cadenas)

In [16]:
with open('matriz_tfidf.pkl', 'wb') as archivo_tfidf:
    pickle.dump(matriz_tfidf, archivo_tfidf)

In [9]:
vocabulario = vectorizer.get_feature_names_out()

In [10]:
from collections import Counter

def obtener_clusters_ordenados(etiquetas):
       # Contar el número de documentos por clúster
    conteo_clusters = Counter(etiquetas)

    # Eliminar el clúster residual (-1)
    del conteo_clusters[-1]

    # Ordenar por número de documentos (de mayor a menor)
    clusters_ordenados = sorted(conteo_clusters.items(), key=lambda x: x[1], reverse=True)

    return clusters_ordenados


In [17]:
def obtener_palabras_representativas_tfidf(matriz_tfidf, etiquetas, vocabulario, n=15,n_top_clusters=20, output_excel_path="palabras_representativas_agg_tfidf.xlsx"):
   
    cluster_palabras_representativas = {}

# Obtener los clústeres ordenados por el número de documentos
    clusters_ordenados = obtener_clusters_ordenados(etiquetas)

    # Tomar los top N clústeres
    top_clusters = clusters_ordenados[:n_top_clusters]

    # Crear una lista para almacenar los datos de las palabras y su clúster
    data = []    
    
    for cluster_label, _ in top_clusters:
        # Obtener los documentos que pertenecen al clúster
        cluster_indices = np.where(etiquetas == cluster_label)[0]
        cluster_tfidf = matriz_tfidf[cluster_indices]

        # Verificar que haya documentos en el clúster
        if cluster_tfidf.shape[0] == 0:
            continue

        # Calcular el promedio de TF-IDF para cada palabra en el clúster
        tfidf_promedio = cluster_tfidf.mean(axis=0).A1  #A1 convierte la matriz sparse a un array 1D eficiente

        # Obtener los índices de las n palabras con la mayor puntuación TF-IDF promedio
        palabras_representativas_indices = np.argsort(tfidf_promedio)[::-1][:n]

        # Obtener las palabras representativas
        palabras_representativas = [vocabulario[i] for i in palabras_representativas_indices]

         # Unir las palabras representativas en una sola cadena separada por comas
        palabras_string = ", ".join(palabras_representativas)

        # Añadir el ID del clúster y las palabras al listado
        data.append({"Cluster ID": cluster_label, "Palabras": palabras_string})

    # Crear un DataFrame de pandas
    df = pd.DataFrame(data)

    # Exportar a Excel
    df.to_excel(output_excel_path, index=False)

    print(f"Las palabras representativas de los clústeres se han exportado a '{output_excel_path}'.")

In [18]:
obtener_palabras_representativas_tfidf(matriz_tfidf, etiquetas_predichas, vocabulario)

Las palabras representativas de los clústeres se han exportado a 'palabras_representativas_agg_tfidf.xlsx'.


In [27]:
#documentos_originales=df_preprocesado['texto']
with bz2.BZ2File('recursos/documentos_originales.pkl.bz2', 'rb') as f:
    documentos_originales = pickle.load(f)

In [19]:
import numpy as np
import pandas as pd
from collections import defaultdict
from numpy.linalg import norm

def imprimir_documentos_relevantes_clusters(etiquetas_predichas, documentos_preprocesados, documentos_originales, top_index, num_documentos=5, nombre_archivo="documentos_relevantes.xlsx"):
    
    resultados_totales = []
 
    for cluster_id in top_index:
        documentos_cluster = [(i, documentos_preprocesados[i]) for i, etiqueta in enumerate(etiquetas_predichas) if etiqueta == cluster_id]

        # Calcular TF
        tf_cluster = defaultdict(int)
        for _, documento in documentos_cluster:
            for palabra in documento:
                tf_cluster[palabra] += 1

        # Calcular IDF
        palabras_por_cluster = defaultdict(set)
        for i, etiqueta in enumerate(etiquetas_predichas):
            palabras_por_cluster[etiqueta].update(documentos_preprocesados[i])

        num_clusters = len(set(etiquetas_predichas))
        num_clusters_con_palabra = {}
        idf_cluster = {}
        peso_palabras = {}
        for palabra in tf_cluster:
            contador = 0
            for palabras in palabras_por_cluster.values():
                if palabra in palabras:
                    contador += 1
            num_clusters_con_palabra[palabra] = contador
            frecuencia = num_clusters_con_palabra[palabra]
            idf = np.log((1 + num_clusters) / (1 + frecuencia)) + 1
            idf_cluster[palabra] = idf
            peso_palabras[palabra] = tf_cluster[palabra] * idf_cluster[palabra]

        # Evaluar documentos del clúster
        puntuaciones_relevancia = []
        for doc_id, documento in documentos_cluster:
            # Calcular la frecuencia relativa de cada palabra
            total_palabras_doc = len(documento)
            puntuacion = np.array([peso_palabras.get(palabra, 0) * (documento.count(palabra) / total_palabras_doc) for palabra in documento])
            puntuacion_final = sum(puntuacion)

            # Cortar los primeros 300 caracteres del documento original
            documento_recortado = documentos_originales[doc_id][:300]

            puntuaciones_relevancia.append((doc_id, cluster_id, documento_recortado, puntuacion_final))

        # Ordenar por puntuación
        puntuaciones_relevancia_ordenadas = sorted(puntuaciones_relevancia, key=lambda x: x[3], reverse=True)[:num_documentos]

        # Añadir resultados de este cluster
        resultados_totales.extend(puntuaciones_relevancia_ordenadas)

    # Verificar que hay datos antes de crear el DataFrame
    if resultados_totales:
        df = pd.DataFrame(resultados_totales, columns=["ID", "ID de Cluster", "Documento (200 caracteres)", "Puntuación"])

        # Guardar en formato CSV, XLSX o ODS según el nombre de archivo
        if nombre_archivo.endswith('.csv'):
            df.to_csv(nombre_archivo, index=False, encoding="utf-8")
        elif nombre_archivo.endswith('.ods'):
            df.to_excel(nombre_archivo, index=False, engine="odf")
        else:
            df.to_excel(nombre_archivo, index=False, engine="openpyxl")
        print(f" Archivo guardado: {nombre_archivo}")
    else:
        print("No hay documentos relevantes para los clusters indicados.")


In [20]:
documentos_originales=df_preprocesado['texto']
# Obtener los clústeres ordenados por el número de documentos
clusters_ordenados = obtener_clusters_ordenados(etiquetas_predichas)

# Tomar los top N clústeres
top_clusters = clusters_ordenados[:20]
top_clusters_ids = [cluster_id for cluster_id, _ in top_clusters]

In [ ]:
imprimir_documentos_relevantes_clusters(etiquetas_predichas, documents, documentos_originales, top_clusters_ids, num_documentos=5, nombre_archivo="documentos_relevantes_agglo_tfidf.xlsx")

# Word2vec

In [23]:
with open('recursos/modelos_topicos/agglomerative_word2vec_ejecucion_0.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)

In [24]:
etiquetas_predichas=resultados["agglomerative_word2vec"]['etiquetas_predichas'][0]

In [7]:
etiquetas_predichas

array([23, 23, 11, ...,  7, 20,  7], dtype=int64)

In [8]:
import numpy as np

def obtener_palabras_representativas_w2v(model_w2v, documents, labels, n=10):
   

    cluster_palabras_representativas = {}

    for cluster_label in np.unique(labels):
        cluster_indices = np.where(labels == cluster_label)[0]
        cluster_documentos = [documents[i] for i in cluster_indices]

        # Obtener embeddings promedio de palabras por documento
        cluster_vectores = []
        for doc in cluster_documentos:
            palabra_vectors = [model_w2v.wv[palabra] for palabra in doc if palabra in model_w2v.wv]
            if palabra_vectors:  # Evitar documentos vacíos
                cluster_vectores.append(np.mean(palabra_vectors, axis=0))

        if not cluster_vectores:
            cluster_palabras_representativas[cluster_label] = []
            continue

        # Calcular el promedio del clúster
        cluster_centroide = np.mean(cluster_vectores, axis=0)

        # Encontrar palabras más similar al promedio
        try:
            palabras_cercanas = model_w2v.wv.similar_by_vector(cluster_centroide, topn=n)
            cluster_palabras_representativas[cluster_label] = [palabra for palabra, _ in palabras_cercanas]
        except:
            cluster_palabras_representativas[cluster_label] = []

    return cluster_palabras_representativas


In [25]:
def filter_documents_for_dictionary(tokenized_documents, filtered_words):
    filtered_documents = []
    for doc in tokenized_documents:
        filtered_doc = [word for word in doc if word in filtered_words]
        filtered_documents.append(filtered_doc)
    return filtered_documents

In [26]:
# df_preprocesado = pd.read_csv('df_preprocesado.csv', sep=";")
# dictionary = Dictionary.load('recursos/dictionary_filtrado_spacy.gensim')
# filtered_words = set(dictionary.token2id.keys())
# preprocesado_documents=df_preprocesado['texto_preprocesado'].apply(ast.literal_eval)
# Generar documentos en igual de condiciones
# documents = filter_documents_for_dictionary(preprocesado_documents,filtered_words)
with open('recursos/documents.pkl', 'rb') as archivo:
    documents = pickle.load(archivo)

In [27]:
import numpy as np

def obtener_palabras_representativas_w2v(model_w2v, documents, labels, n=15,n_top_clusters=20,output_excel_path="palabras_representativas_agglo_w2v.xlsx"):
   

    # Obtener los clústeres ordenados por el número de documentos
    clusters_ordenados = obtener_clusters_ordenados(labels)

    # Tomar los top N clústeres
    top_clusters = clusters_ordenados[:n_top_clusters]
    cluster_palabras_representativas = {}
# Crear una lista para almacenar los datos de las palabras y su clúster
    data = []
    for cluster_label, _ in top_clusters:
        cluster_indices = np.where(labels == cluster_label)[0]
        cluster_documentos = [documents[i] for i in cluster_indices]

        # Obtener embeddings promedio de palabras por documento
        cluster_vectores = []
        for doc in cluster_documentos:
            palabra_vectors = [model_w2v.wv[palabra] for palabra in doc if palabra in model_w2v.wv]
            if palabra_vectors:  # Evitar documentos vacíos
                cluster_vectores.append(np.mean(palabra_vectors, axis=0))

        if not cluster_vectores:
            cluster_palabras_representativas[cluster_label] = []
            continue

        # Calcular el promedio del clúster
        cluster_centroide = np.mean(cluster_vectores, axis=0)

        # Encontrar palabras más similar al promedio
        try:
            palabras_cercanas = model_w2v.wv.similar_by_vector(cluster_centroide, topn=n)
            cluster_palabras_representativas[cluster_label] = [palabra for palabra, _ in palabras_cercanas]
        except:
            cluster_palabras_representativas[cluster_label] = []

    # Unir las palabras representativas en una sola cadena separada por comas
        palabras_string = ", ".join(cluster_palabras_representativas[cluster_label])

        # Añadir el ID del clúster y las palabras al listado
        data.append({"Cluster ID": cluster_label, "Palabras": palabras_string})

    # Crear un DataFrame de pandas
    df = pd.DataFrame(data)

    # Exportar a Excel
    df.to_excel(output_excel_path, index=False)

    print(f"Las palabras representativas de los clústeres se han exportado a '{output_excel_path}'.")

In [28]:
# Cargar el modelo Word2Vec y las etiquetas predichas

modelo_w2v_cargado = Word2Vec.load("recursos/w2v_250_epochs.model")


# Obtener las palabras representativas
palabras_representativas = obtener_palabras_representativas_w2v(modelo_w2v_cargado, documents, etiquetas_predichas)

Las palabras representativas de los clústeres se han exportado a 'palabras_representativas_agglo_w2v.xlsx'.


In [29]:
# df_preprocesado = pd.read_csv('df_preprocesado.csv', sep=';')
# documentos_originales=df_preprocesado['texto']
with bz2.BZ2File('recursos/documentos_originales.pkl.bz2', 'rb') as f:
    documentos_originales = pickle.load(f)
# Obtener los clústeres ordenados por el número de documentos
clusters_ordenados = obtener_clusters_ordenados(etiquetas_predichas)

# Tomar los top N clústeres
top_clusters = clusters_ordenados[:20]
top_clusters_ids = [cluster_id for cluster_id, _ in top_clusters]

In [ ]:
imprimir_documentos_relevantes_clusters(etiquetas_predichas, documents, documentos_originales, top_clusters_ids, num_documentos=5, nombre_archivo="documentos_relevantes_agglo_word2vec.xlsx")